# HowLongToBeat RDF Creator

We load the generated CSV files and we serialize all the data into ***turtle format  (TTL)*** relying on ***RDFLib*** Python library.

## Setup

We import all the necessary libraries and we set the paths to the input/output files. In particular, we create a TTL file for each type of data.

In [32]:
# Imports
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# RDFLib knows about some namespaces, like XSD
from rdflib.namespace import XSD

In [33]:
absPath = str(Path(os.path.abspath(os.getcwd())).absolute())
datasetsPath = os.path.join(absPath, "cleaned_datasets")
rawdatasetsPath = os.path.join(absPath, "raw_datasets")
rdfPath = os.path.join(absPath, "rdf")

# Create dataset directory if not exists
if not os.path.exists(datasetsPath):
    os.mkdir(datasetsPath)

# Create RDF directory if not exists
if not os.path.exists(rdfPath):
    os.mkdir(rdfPath)

# Setup datasets paths
gamesPath = os.path.join(datasetsPath, "games_cleaned.csv")
vgchartzPath = os.path.join(datasetsPath, "vgchartz_cleaned.csv")
indiegamesdevelopersPath = os.path.join(datasetsPath, "indiegamesdevelopers_cleaned_seriesExplode.csv")
platformsPath = os.path.join(datasetsPath, "platforms.csv")
videoGameDevelopersPath = os.path.join(datasetsPath, "videogamesdevelopers_cleaned_seriesexplode.csv")
completionTimePath = os.path.join(datasetsPath, "completion_time.csv")

# Setup raw datasets
rawVGChartsPath = os.path.join(rawdatasetsPath, "vgchartz-7_7_2020.csv")
rawGamesPath = os.path.join(rawdatasetsPath, "games.csv")

# Countries-Regions path
countriesRegionsPath = os.path.join(datasetsPath, "countries-regions.csv")
countriesPath = os.path.join(absPath, "wikipedia-iso-country-codes.csv")

# Setup Turtle paths
genresTTLPath = os.path.join(rdfPath, "genres.ttl")
gamesTTLPath = os.path.join(rdfPath, "games.ttl")
companyTTLPath = os.path.join(rdfPath, "company.ttl")
platformsTTLPath = os.path.join(rdfPath, "platforms.ttl")
platformsSalesTTLPath = os.path.join(rdfPath, "platformsSales.ttl")
videoGameDevelopersTTLPath = os.path.join(rdfPath, "videoGameDevelopers.ttl")
statsTTLPath = os.path.join(rdfPath, "stats.ttl")
gameSalesTTLPath = os.path.join(rdfPath, "gameSales.ttl")
regionsTTLPath = os.path.join(rdfPath, "regions.ttl")
countriesTTLPath = os.path.join(rdfPath, "countries.ttl")


In [34]:
# Country Ontology
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")

# HLTB Ontology
HLTB = Namespace("http://www.semanticweb.org/enrico/ontologies/2022/10/HLTB-db2unipd#")

In [35]:
def createGraph():
    # Create the graph
    g = Graph()

    # Bind the namespaces to a prefix for more readable output
    g.bind("xsd", XSD)
    g.bind("countries", CNS)
    g.bind("hltb", HLTB)

    return g


#create game URI
def createGameID(title):
    # Replace all special chars with "-"
    gameID = ""
    for char in title:
        if char.isalnum():
            gameID += char
        elif len(gameID) > 0 and gameID[-1] != '-':
            gameID += '-'
    if len(gameID) > 0 and gameID[-1] == '-':
        gameID = gameID[:-1]
    #print(gameID.lower())
    return gameID.lower()


#Create genre URI
def setGenreID(genre):  ##first half is the original genres, second half are processed and lowercase
    genre = str(genre).replace("/", ", ").replace("nan", "")
    genre = genre.split(", ")
    list = []
    for i in range(len(genre)):
        list.append([])
        list[i].append(genre[i])
        list[i].append(genre[i].lower().replace("'", "").replace(" ", "-"))
    return (list)


def setPlatformID(platform):
    return platform.lower().replace(" ", "-").replace("/", "-").replace("&", "-")


def setCompanyID(company):
    if ' ' in company:
        return company.replace(' ', '-').lower()
    return company.lower()


def setDeveloperID(developer):
    return developer.replace(' ', '-', regex=True).str.lower()


def setCountryID(country):
    return country.replace(' ', '-', regex=True).str.lower()


## Serialization

We serialize the data according to the following workflow:

1. Load the CSV file and iterate through it
2. Create a unique ID by ourself based on the name of the class.
3. Add the node to the graph using the unique ID.
4. Add all the data properties.
5. Add all the object properties.
6. Serialize the data and save them into a TTL file.

### Games

Now serializing the Game class

In [36]:
# Create Graph
g = createGraph()

In [37]:
# Load the CSV files in memory
games = pd.read_csv(gamesPath, sep=",", index_col="title")
vgchartz = pd.read_csv(vgchartzPath, sep=",")
platforms = pd.read_csv(platformsPath, sep=",")

merged = pd.merge(vgchartz, platforms, left_on='console', right_on='Acronym', how='left')

In [38]:
# Iterate over the games
for title, row in games.iterrows():
    # Create gameID from its title
    gameID = createGameID(title)

    # Create the node to add to the Graph
    Game = URIRef(HLTB[gameID])

    # Add triples using store's add() method.
    g.add((Game, RDF.type, HLTB.Game))

    # Add the title of the game
    g.add((Game, HLTB["officialName"], Literal(title, datatype=XSD.string)))

    # Add multiplayer focus
    g.add((Game, HLTB["multiplayerFocus"],
           Literal(pd.notnull(row["coop"]) or pd.notnull(row["versus"]), datatype=XSD.boolean)))

    #Add hltb id
    g.add((Game, HLTB["id"], Literal(row["id"], datatype=XSD.int)))

    #Add hasGenre object property
    for iterator in setGenreID(row["genres"]):
        if pd.notnull(iterator[1]) and iterator[1] != '':
            g.add((Game, HLTB["hasGenre"], URIRef(HLTB[iterator[1]])))

    #Add platform availability
    if pd.notna(row["platforms"]):
        for platform in row["platforms"].split(", "):
            g.add((Game, HLTB["releasedOn"], URIRef(HLTB[setPlatformID(platform)])))

            #Add Stats object property
            g.add((Game, HLTB["hasStats"],
                   URIRef(HLTB["stats-" + str(createGameID(title)) + "___" + str(setPlatformID(platform))])))

            # Add Sales object property
            game = merged.loc[(merged['title'] == title) & (merged['Platform'] == platform)]
            if not game.empty:
                if pd.notna(game["pal_sales"].iloc[0]):
                    GameSalesID = URIRef(
                        HLTB["sales-" + str(createGameID(title)) + "___" + str(setPlatformID(platform)) + "___" + "eu"])
                    g.add((Game, HLTB["sold"], GameSalesID))
                if pd.notna(game["na_sales"].iloc[0]):
                    GameSalesID = URIRef(
                        HLTB["sales-" + str(createGameID(title)) + "___" + str(setPlatformID(platform)) + "___" + "na"])
                    g.add((Game, HLTB["sold"], GameSalesID))
                if pd.notna(game["jp_sales"].iloc[0]):
                    GameSalesID = URIRef(
                        HLTB["sales-" + str(createGameID(title)) + "___" + str(setPlatformID(platform)) + "___" + "jp"])
                    g.add((Game, HLTB["sold"], GameSalesID))
                if pd.notna(game["other_sales"].iloc[0]):
                    GameSalesID = URIRef(HLTB["sales-" + str(createGameID(title)) + "___" + str(
                        setPlatformID(platform)) + "___" + "other"])
                    g.add((Game, HLTB["sold"], GameSalesID))
                if pd.notna(game["total_shipped"].iloc[0]):
                    GameSalesID = URIRef(HLTB["sales-" + str(createGameID(title)) + "___" + str(
                        setPlatformID(platform)) + "___" + "global"])
                    g.add((Game, HLTB["sold"], GameSalesID))

In [39]:
# Save the data in the Turtle format
with open(gamesTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

print("Saved games TTL file.")

Saved games TTL file.


### Genre

Now serializing the Genre class

In [40]:
# Create Graph
g = createGraph()

# Load the CSV files in memory
genres = pd.read_csv(gamesPath, sep=",", index_col="genres")

In [41]:
for genre, row in genres.iterrows():
    allGenres = setGenreID(genre)
    for iterator in allGenres:
        if not (iterator[0] == " " or iterator[0] == ""):
            Genre = URIRef(HLTB[iterator[1]])
            #Add triples using store's add() method.
            g.add((Genre, RDF.type, HLTB.Genre))
            # Add the name of the genre
            g.add((Genre, HLTB["name"], Literal(iterator[0], datatype=XSD.string)))

In [42]:
# Save genre data in the Turtle format
with open(genresTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

### Platforms

In [43]:
# Create Graph
g = createGraph()
# Load the CSV files in memory
platforms = pd.read_csv(platformsPath, sep=",", index_col="Platform")

In [44]:
manufacturerDict = {}

for platform, row in platforms.iterrows():
    Platform = URIRef(HLTB[setPlatformID(platform)])
    #Add triples using store's add() method.
    g.add((Platform, RDF.type, HLTB.Platform))
    # Add the name of the genre
    g.add((Platform, HLTB["name"], Literal(platform, datatype=XSD.string)))

    #Add popularity if platform is popular
    if row["Popular"]:
        g.add((Platform, HLTB["popular"], Literal(True, datatype=XSD.boolean)))

    #Add release date if present
    if pd.notna(row["Release date"]):
        time = datetime.combine(datetime.strptime(row["Release date"], '%Y-%M-%d'), datetime.min.time())
        g.add((Platform, HLTB["releaseDate"], Literal(time, datatype=XSD.dateTime)))

    #Add CPU information
    if pd.notna(row["CPU"]):
        g.add((Platform, HLTB["cpu"], Literal(row["CPU"], datatype=XSD.string)))

    #Add CPU bit information
    if pd.notna(row["\"Bits\""]):
        bits = row["\"Bits\""].split("-")[0]
        g.add((Platform, HLTB["bits"], Literal(bits, datatype=XSD.int)))

    #Add acronym information
    if pd.notna(row["Acronym"]):
        g.add((Platform, HLTB["acronym"], Literal(row["Acronym"], datatype=XSD.string)))

    # add the manufacturer name
    if pd.notna(row["Manufacturer"]) or pd.notnull(row["Manufacturer"]):
        manufacturerStr = row["Manufacturer"].lstrip().rstrip()
        if ',' in manufacturerStr:
            manufacturerStr = manufacturerStr.replace(',', '/')
        manufacturerSplit = manufacturerStr.split('/')
        for elem in manufacturerSplit:
            elem = elem.lstrip()
            manufacturerName = ""
            manufacturerCountry = ""
            if '(' in elem:
                elemSplit = elem.split('(')
                manufacturerName = elemSplit[0].lstrip().rstrip()
                manufacturerCountry = elemSplit[1].lstrip().rstrip()
                if not manufacturerName[-1].isalnum():
                    manufacturerName = manufacturerName[:-1]
                if manufacturerCountry[-1] == ')':
                    manufacturerCountry = manufacturerCountry[:-1]
                manufacturerDict[manufacturerName] = manufacturerCountry
            if manufacturerName != "" and manufacturerCountry != "":
                manufacturer = URIRef(HLTB[setCompanyID(manufacturerName)])
                g.add((Platform, HLTB["createdBy"], manufacturer))
            # match manufacturer with companies
print(manufacturerDict)

{'Panasonic': 'Japan', 'Sanyo': 'Japan', 'GoldStar': 'South Korea', 'Commodore': 'Canada', 'Atari Inc': 'U.S.', 'Atari Corporation': 'U.S.', 'Coleco': 'U.S.', 'Sega': 'Japan', 'Fujitsu': 'Japan', 'Mattel Electronics': 'U.S.', 'Nintendo': 'Japan', 'Magnavox': 'U.S.', 'Philips': 'Netherlands', 'Sony': 'Japan', 'Tec Toy': 'Brazil', 'Majesco Entertainment': 'Japan', 'Microsoft': 'U.S.', 'Zeebo Inc': 'U.S.'}


In [45]:
# Save data in the Turtle format
with open(platformsTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

### Stats


In [46]:
g = createGraph()
games = pd.read_csv(gamesPath, sep=",")
vgchartz = pd.read_csv(vgchartzPath, sep=",")
completionTime = pd.read_csv(completionTimePath, sep=",")
platforms = pd.read_csv(platformsPath, sep=",")

merged_temp = pd.merge(games, completionTime, left_on='id',
                       right_on='gameID')
merged_temp1 = pd.merge(vgchartz, platforms, left_on='console', right_on='Acronym', how='left')
merged = pd.merge(merged_temp, merged_temp1, left_on=["title", "platform"], right_on=["title", "Platform"], how='left')

In [47]:
for id, row in merged.iterrows():
    if pd.notna(row["title"]) and pd.notna(row["platform"]):
        StatsID = URIRef(HLTB[createGameID("stats-" + str(createGameID(row["title"]))) + "___" + str(
            setPlatformID(row["platform"]))])

        #Adding node type
        g.add((StatsID, HLTB.Type, HLTB.Stats))

        #Add Time information
        g.add((StatsID, HLTB["polledTime"], Literal(row["count_comp"], datatype=XSD.int)))
        g.add((StatsID, HLTB["mainTime"], Literal(row["comp_main"], datatype=XSD.int)))
        g.add((StatsID, HLTB["mainPlusTime"], Literal(row["comp_plus"], datatype=XSD.int)))
        g.add((StatsID, HLTB["completionistTime"], Literal(row["comp_100"], datatype=XSD.int)))
        g.add((StatsID, HLTB["slowestTime"], Literal(row["comp_low"], datatype=XSD.int)))
        g.add((StatsID, HLTB["fastestTime"], Literal(row["comp_high"], datatype=XSD.int)))

        #Add remaining stats
        if pd.notna(row["critic_score"]):
            g.add((StatsID, HLTB["criticScore"], Literal(row["critic_score"], datatype=XSD.float)))
        if pd.notna(row["user_score"]):
            g.add((StatsID, HLTB["userScore"], Literal(row["user_score"], datatype=XSD.float)))

        if pd.notna(row["release_date"]):
            time = datetime.combine(datetime.strptime(row["release_date"], '%Y-%M-%d'), datetime.min.time())
            g.add((StatsID, HLTB["releaseDate"], Literal(time, datatype=XSD.dateTime)))

        #Add "onPlatform" object property
        g.add((StatsID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["platform"])])))

In [48]:

# Save the data in the Turtle format
with open(statsTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

print("Saved company TTL file.")

Saved company TTL file.


### Game sales

In [49]:
g = createGraph()
vgchartz = pd.read_csv(vgchartzPath, sep=",")
platforms = pd.read_csv(platformsPath, sep=",")

merged = pd.merge(vgchartz, platforms, left_on='console', right_on='Acronym', how='left')
merged.to_csv("prova.csv")

In [50]:
for id, row in merged.iterrows():
    if pd.notna(row["Platform"]):

        if pd.notna(row["pal_sales"]):
            GameSalesID = URIRef(HLTB["sales-" + str(createGameID(row["title"])) + "___" + str(
                setPlatformID(row["Platform"])) + "___" + "eu"])
            g.add((GameSalesID, HLTB.Type, HLTB.Sale))
            g.add((GameSalesID, HLTB["unitsSold"], Literal(row["pal_sales"], datatype=XSD.float)))
            g.add((GameSalesID, HLTB["locatedIn"], URIRef(HLTB["eu"])))
            g.add((GameSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["jp_sales"]):
            GameSalesID = URIRef(HLTB["sales-" + str(createGameID(row["title"])) + "___" + str(
                setPlatformID(row["Platform"])) + "___" + "jp"])
            g.add((GameSalesID, HLTB.Type, HLTB.Sale))
            g.add((GameSalesID, HLTB["unitsSold"], Literal(row["jp_sales"], datatype=XSD.float)))
            g.add((GameSalesID, HLTB["locatedIn"], URIRef(HLTB["jp"])))
            g.add((GameSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["na_sales"]):
            GameSalesID = URIRef(HLTB["sales-" + str(createGameID(row["title"])) + "___" + str(
                setPlatformID(row["Platform"])) + "___" + "na"])
            g.add((GameSalesID, HLTB.Type, HLTB.Sale))
            g.add((GameSalesID, HLTB["unitsSold"], Literal(row["na_sales"], datatype=XSD.float)))
            g.add((GameSalesID, HLTB["locatedIn"], URIRef(HLTB["na"])))
            g.add((GameSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["other_sales"]):
            GameSalesID = URIRef(HLTB["sales-" + str(createGameID(row["title"])) + "___" + str(
                setPlatformID(row["Platform"])) + "___" + "other"])
            g.add((GameSalesID, HLTB.Type, HLTB.Sale))
            g.add((GameSalesID, HLTB["unitsSold"], Literal(row["other_sales"], datatype=XSD.float)))
            g.add((GameSalesID, HLTB["locatedIn"], URIRef(HLTB["other"])))
            g.add((GameSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["total_shipped"]):
            GameSalesID = URIRef(HLTB["sales-" + str(createGameID(row["title"])) + "___" + str(
                setPlatformID(row["Platform"])) + "___" + "global"])
            g.add((GameSalesID, HLTB.Type, HLTB.Sale))
            g.add((GameSalesID, HLTB["unitsSold"], Literal(row["total_shipped"], datatype=XSD.float)))
            g.add((GameSalesID, HLTB["locatedIn"], URIRef(HLTB["global"])))
            g.add((GameSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

In [51]:
# Save the data in the Turtle format
with open(gameSalesTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

print("Saved game-sales TTL file.")

Saved game-sales TTL file.


### Company

In [52]:
# Create Graph
g = createGraph()
# Load the CSV files in memory

indiegamesdevelopers = pd.read_csv(indiegamesdevelopersPath, sep=",")

In [53]:
#companies=[]
#replace space with -, lower case
indiegamesdevelopers['companyID'] = setDeveloperID(indiegamesdevelopers['Developer'])
#indiegamesdevelopers['companyID'] = setCompanyID(indiegamesdevelopers['Developer'])


#indiegamesdevelopers.info()
for index, row in indiegamesdevelopers.iterrows():
    # Create the node to add to the Graph
    Company = URIRef(HLTB[row['companyID']])
    #companies.append(Company)

    # Add triples using store's add() method.
    g.add((Company, RDF.type, HLTB.Company))

    # Add the Company
    g.add((Company, HLTB["indieDeveloper"], Literal(pd.notnull(row['Developer']), datatype=XSD.boolean)))
    g.add((Company, HLTB['officialName'], Literal(row['Developer'], datatype=XSD.string)))


### video-game-developers

In [54]:
videoGameDevelopers = pd.read_csv(videoGameDevelopersPath, sep=",")

In [55]:
#replace space with -, lower case
videoGameDevelopers['companyID'] = setDeveloperID(videoGameDevelopers['Developer'])
#videoGameDevelopers['companyID'] = setCompanyID(videoGameDevelopers['Developer'])


#indiegamesdevelopers.info()
for index, row in videoGameDevelopers.iterrows():
    # Create the node to add to the Graph
    Company = URIRef(HLTB[row['companyID']])
    #companies.append(Company)

    # Add triples using store's add() method.
    g.add((Company, RDF.type, HLTB.Company))

    # Add the Company
    g.add((Company, HLTB['officialName'], Literal(row['Developer'], datatype=XSD.string)))


In [56]:
# Save the data in the Turtle format
with open(companyTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

print("Saved company TTL file.")

Saved company TTL file.


Region

In [57]:
# Create Graph
g = createGraph()

In [58]:
# Create the node to add to the Graph
Region = URIRef(HLTB['eu'])
g.add((Region, RDF.type, HLTB.Region))
g.add((Region, HLTB[{'eu', 'jp', 'na', 'other', 'global'}], Literal('eu', datatype=XSD.string)))

Region = URIRef(HLTB['jp'])
g.add((Region, RDF.type, HLTB.Region))
g.add((Region, HLTB[{'eu', 'jp', 'na', 'other', 'global'}], Literal('jp', datatype=XSD.string)))

Region = URIRef(HLTB['na'])
g.add((Region, RDF.type, HLTB.Region))
g.add((Region, HLTB[{'eu', 'jp', 'na', 'other', 'global'}], Literal('na', datatype=XSD.string)))

Region = URIRef(HLTB['other'])
g.add((Region, RDF.type, HLTB.Region))
g.add((Region, HLTB[{'eu', 'jp', 'na', 'other', 'global'}], Literal('other', datatype=XSD.string)))


<Graph identifier=Ne85290bb3552426ca0160dbe98d87376 (<class 'rdflib.graph.Graph'>)>

In [59]:
# Save the data in the Turtle format
with open(regionsTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

print("Saved Regions TTL file.")

Saved Regions TTL file.


Country

In [60]:
# Load the CSV files in memory
countries = pd.read_csv(countriesPath, sep=',', index_col='Name', keep_default_na=False, na_values=['_'])
countriesregions = pd.read_csv(countriesRegionsPath, sep=",", names=['Country', 'Region'])

g = createGraph()

In [61]:
countriesregions['countryID'] = setCountryID(countriesregions['Country'])

#countriesregions.info()
for index, row in countriesregions.iterrows():

    for c in str(row['Country']).split(','):
        cName = c.strip()

        if (countries.index == cName).any():
            code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
            Country = URIRef(CNS[row['countryID']])
            g.add((Country, RDF.type, HLTB.Country))
            g.add((Country, HLTB['subClassOf'], Literal(row['Region'], datatype=XSD.string)))



In [62]:
# Save the data in the Turtle format
with open(countriesTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

print("Saved Regions TTL file.")

Saved Regions TTL file.


### Platform sales

In [63]:
g = createGraph()
platforms = pd.read_csv(platformsPath, sep=",")


In [64]:
for id, row in platforms.iterrows():
    if pd.notna(row["Platform"]):

        if pd.notna(row["Europe"]):
            PlatformSalesID = URIRef(HLTB["sales-" + str(setPlatformID(row["Platform"])) + "___" + "eu"])
            g.add((PlatformSalesID, HLTB.Type, HLTB.Sale))
            g.add((PlatformSalesID, HLTB["unitsSold"], Literal(row["Europe"], datatype=XSD.float)))
            g.add((PlatformSalesID, HLTB["locatedIn"], URIRef(HLTB["eu"])))
            g.add((PlatformSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["Japan"]):
            PlatformSalesID = URIRef(HLTB["sales-" + str(setPlatformID(row["Platform"])) + "___" + "jp"])

            g.add((PlatformSalesID, HLTB.Type, HLTB.Sale))
            g.add((PlatformSalesID, HLTB["unitsSold"], Literal(row["Japan"], datatype=XSD.float)))
            g.add((PlatformSalesID, HLTB["locatedIn"], URIRef(HLTB["jp"])))
            g.add((PlatformSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["North America"]):
            PlatformSalesID = URIRef(HLTB["sales-" + str(setPlatformID(row["Platform"])) + "___" + "na"])

            g.add((PlatformSalesID, HLTB.Type, HLTB.Sale))
            g.add((PlatformSalesID, HLTB["unitsSold"], Literal(row["North America"], datatype=XSD.float)))
            g.add((PlatformSalesID, HLTB["locatedIn"], URIRef(HLTB["na"])))
            g.add((PlatformSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["Rest of World"]):
            PlatformSalesID = URIRef(HLTB["sales-" + str(setPlatformID(row["Platform"])) + "___" + "other"])

            g.add((PlatformSalesID, HLTB.Type, HLTB.Sale))
            g.add((PlatformSalesID, HLTB["unitsSold"], Literal(row["Rest of World"], datatype=XSD.float)))
            g.add((PlatformSalesID, HLTB["locatedIn"], URIRef(HLTB["other"])))
            g.add((PlatformSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

        if pd.notna(row["Global"]):
            PlatformSalesID = URIRef(HLTB["sales-" + str(setPlatformID(row["Platform"])) + "___" + "global"])
            g.add((PlatformSalesID, HLTB.Type, HLTB.Sale))
            g.add((PlatformSalesID, HLTB["unitsSold"], Literal(row["Global"], datatype=XSD.float)))
            g.add((PlatformSalesID, HLTB["locatedIn"], URIRef(HLTB["global"])))
            g.add((PlatformSalesID, HLTB["onPlatform"], URIRef(HLTB[setPlatformID(row["Platform"])])))

In [65]:
# Save the data in the Turtle format
with open(platformsSalesTTLPath, "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

print("Saved platforms Sales TTL file.")

Saved platforms Sales TTL file.
